In [11]:
import pandas as pd
import requests
import os
import re
import numpy as np

pd.options.display.max_columns = 999
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [12]:
if not os.path.exists('./nfl_data/weekly/weather'):
    os.mkdir('./nfl_data/weekly/weather')

In [13]:
weeks = [f'week-{week}' for week in range(1,17)]

file_list = []

for week in weeks:
    url = f'https://www.nflweather.com/en/week/2021/{week}/'
    print(url)
    res = requests.get(url)
    print(res)
    output = open(f'./nfl_data/weekly/weather/{week}_weather.xls', 'wb')
    output.write(res.content)
    file_list.append(f'./nfl_data/weekly/weather/{week}_weather.xls')
    output.close()
    data = pd.read_html(f'./nfl_data/weekly/weather/{week}_weather.xls')
    df = data[0]
    # df = df.droplevel(0, axis=1)
    print(week, df.shape)

https://www.nflweather.com/en/week/2021/week-1/
<Response [200]>
week-1 (16, 13)
https://www.nflweather.com/en/week/2021/week-2/
<Response [200]>
week-2 (16, 13)
https://www.nflweather.com/en/week/2021/week-3/
<Response [200]>
week-3 (16, 13)
https://www.nflweather.com/en/week/2021/week-4/
<Response [200]>
week-4 (16, 13)
https://www.nflweather.com/en/week/2021/week-5/
<Response [200]>
week-5 (16, 13)
https://www.nflweather.com/en/week/2021/week-6/
<Response [200]>
week-6 (14, 13)
https://www.nflweather.com/en/week/2021/week-7/
<Response [200]>
week-7 (13, 13)
https://www.nflweather.com/en/week/2021/week-8/
<Response [200]>
week-8 (15, 13)
https://www.nflweather.com/en/week/2021/week-9/
<Response [200]>
week-9 (14, 13)
https://www.nflweather.com/en/week/2021/week-10/
<Response [200]>
week-10 (14, 13)
https://www.nflweather.com/en/week/2021/week-11/
<Response [200]>
week-11 (15, 13)
https://www.nflweather.com/en/week/2021/week-12/
<Response [200]>
week-12 (15, 13)
https://www.nflweather

In [14]:
file_list

['./nfl_data/weekly/weather/week-1_weather.xls',
 './nfl_data/weekly/weather/week-2_weather.xls',
 './nfl_data/weekly/weather/week-3_weather.xls',
 './nfl_data/weekly/weather/week-4_weather.xls',
 './nfl_data/weekly/weather/week-5_weather.xls',
 './nfl_data/weekly/weather/week-6_weather.xls',
 './nfl_data/weekly/weather/week-7_weather.xls',
 './nfl_data/weekly/weather/week-8_weather.xls',
 './nfl_data/weekly/weather/week-9_weather.xls',
 './nfl_data/weekly/weather/week-10_weather.xls',
 './nfl_data/weekly/weather/week-11_weather.xls',
 './nfl_data/weekly/weather/week-12_weather.xls',
 './nfl_data/weekly/weather/week-13_weather.xls',
 './nfl_data/weekly/weather/week-14_weather.xls',
 './nfl_data/weekly/weather/week-15_weather.xls',
 './nfl_data/weekly/weather/week-16_weather.xls']

In [15]:
# get df started with the 1st file 'week-1_weather.xls'.  This way we have something to merge to.

data = pd.read_html('./nfl_data/weekly/weather/week-1_weather.xls')
weather_df = pd.DataFrame(data[0]) # Saves df var to dataframe
weather_df.drop(columns = ['Unnamed: 0', 'Game', 'Game.1', 'Game.2', 'Time (ET)', 'TV', 'Unnamed: 8', 'Unnamed: 12'], inplace = True) 
weather_df['Week'] = re.search(r'(\d+)', file_list[0])[0]
weather_df['Wind_Speed_MPH'] = weather_df['Wind'].map(lambda x:re.search(r'(\d+)',x)[0])
weather_df['Wind_Direction'] = weather_df['Wind'].map(lambda x:re.search(r'[A-Z]+',x)[0])
weather_df['Temp'] = weather_df['Forecast'].map(lambda x: re.search(r'(\d+)',x)[0] if x != 'DOME' else 0)
weather_df['Weather_Desc'] = weather_df['Forecast'].map(lambda x: re.search(r'\s(.+)',x)[0].strip() if x != 'DOME' else 0)
weather_df["Wind"] = np.where(weather_df["Forecast"] == "DOME", 0, weather_df["Wind"])
weather_df["Wind_Speed_MPH"] = np.where(weather_df["Forecast"] == "DOME", 0, weather_df["Wind_Speed_MPH"])
weather_df["Wind_Direction"] = np.where(weather_df["Forecast"] == "DOME", 0, weather_df["Wind_Direction"])
weather_df


,Away,Home,Forecast,Extended Forecast,Wind,Week,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc
0,Cowboys,Buccaneers,80f Humid and Partly Cloudy,Humid and Partly Cloudy. Rain until evening.,6m WSW,1,6,WSW,80,Humid and Partly Cloudy
1,Eagles,Falcons,DOME,Clear. Clear throughout the day.,0,1,0,0,0,0
2,Steelers,Bills,72f Mostly Cloudy,Mostly Cloudy. Rain overnight.,12m WSW,1,12,WSW,72,Mostly Cloudy
3,Vikings,Bengals,82f Clear,Clear. Clear throughout the day.,11m SW,1,11,SW,82,Clear
4,49ers,Lions,DOME,Mostly Cloudy. Mostly cloudy throughout the day.,0,1,0,0,0,0
5,Cardinals,Titans,82f Clear,Clear. Clear throughout the day.,6m SSW,1,6,SSW,82,Clear
6,Seahawks,Colts,DOME,Clear. Clear throughout the day.,0,1,0,0,0,0
7,Chargers,Washington,82f Clear,Clear. Clear throughout the day.,8m SW,1,8,SW,82,Clear
8,Jets,Panthers,82f Clear,Clear. Clear throughout the day.,4m WSW,1,4,WSW,82,Clear
9,Jaguars,Texans,87f Partly Cloudy,Partly Cloudy. Rain starting in the afternoon.,8m E,1,8,E,87,Partly Cloudy


In [16]:
weather_df.shape

(16, 10)

In [17]:
for file in file_list[1:]:
    print(file)
    data = pd.read_html(file)
    df = pd.DataFrame(data[0])
    df.drop(columns = ['Unnamed: 0', 'Game', 'Game.1', 'Game.2', 'Time (ET)', 'TV', 'Unnamed: 8', 'Unnamed: 12'], inplace = True) 
    df['Week'] = re.search(r'(\d+)', file)[0]
    df['Wind_Speed_MPH'] = df['Wind'].map(lambda x:re.search(r'(\d+)',x)[0])
    df['Wind_Direction'] = df['Wind'].map(lambda x:re.search(r'[A-Z]+',x)[0])
    df['Temp'] = df['Forecast'].map(lambda x: re.search(r'(\d+)',x)[0] if x != 'DOME' else 0)
    df['Weather_Desc'] = df['Forecast'].map(lambda x: re.search(r'\s(.+)',x)[0].strip() if x != 'DOME' else 0)
    df["Wind"] = np.where(df["Forecast"] == "DOME", 0, df["Wind"])
    df["Wind_Speed_MPH"] = np.where(df["Forecast"] == "DOME", 0, df["Wind_Speed_MPH"])
    df["Wind_Direction"] = np.where(df["Forecast"] == "DOME", 0, df["Wind_Direction"])
    dfs = [weather_df, df]
    weather_df = pd.concat([weather_df.squeeze() for weather_df in dfs], ignore_index=True)

print(f'The shape of the updated df is {weather_df.shape}')

./nfl_data/weekly/weather/week-2_weather.xls
./nfl_data/weekly/weather/week-3_weather.xls
./nfl_data/weekly/weather/week-4_weather.xls
./nfl_data/weekly/weather/week-5_weather.xls
./nfl_data/weekly/weather/week-6_weather.xls
./nfl_data/weekly/weather/week-7_weather.xls
./nfl_data/weekly/weather/week-8_weather.xls
./nfl_data/weekly/weather/week-9_weather.xls
./nfl_data/weekly/weather/week-10_weather.xls
./nfl_data/weekly/weather/week-11_weather.xls
./nfl_data/weekly/weather/week-12_weather.xls
./nfl_data/weekly/weather/week-13_weather.xls
./nfl_data/weekly/weather/week-14_weather.xls
./nfl_data/weekly/weather/week-15_weather.xls
./nfl_data/weekly/weather/week-16_weather.xls
The shape of the updated df is (240, 10)


In [18]:
weather_df.head(15)

,Away,Home,Forecast,Extended Forecast,Wind,Week,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc
0,Cowboys,Buccaneers,80f Humid and Partly Cloudy,Humid and Partly Cloudy. Rain until evening.,6m WSW,1,6,WSW,80,Humid and Partly Cloudy
1,Eagles,Falcons,DOME,Clear. Clear throughout the day.,0,1,0,0,0,0
2,Steelers,Bills,72f Mostly Cloudy,Mostly Cloudy. Rain overnight.,12m WSW,1,12,WSW,72,Mostly Cloudy
3,Vikings,Bengals,82f Clear,Clear. Clear throughout the day.,11m SW,1,11,SW,82,Clear
4,49ers,Lions,DOME,Mostly Cloudy. Mostly cloudy throughout the day.,0,1,0,0,0,0
5,Cardinals,Titans,82f Clear,Clear. Clear throughout the day.,6m SSW,1,6,SSW,82,Clear
6,Seahawks,Colts,DOME,Clear. Clear throughout the day.,0,1,0,0,0,0
7,Chargers,Washington,82f Clear,Clear. Clear throughout the day.,8m SW,1,8,SW,82,Clear
8,Jets,Panthers,82f Clear,Clear. Clear throughout the day.,4m WSW,1,4,WSW,82,Clear
9,Jaguars,Texans,87f Partly Cloudy,Partly Cloudy. Rain starting in the afternoon.,8m E,1,8,E,87,Partly Cloudy


In [19]:
weather_df.to_csv(f'./nfl_data/weekly/weather/all_weeks_weather.csv', index=0)